In [1]:
import scipy.io as spio
import numpy as np
import datetime
import random
from collections import Counter

In [2]:
def l2_norm(x):
    return x / (np.linalg.norm(x) + 10e-6)

In [3]:
path="/media/batman/Windows/linuxshared/xlsa17/data/"

In [4]:
image_info = spio.loadmat(r"/media/batman/Windows/linuxshared/xlsa17/data/AWA2/res101.mat")
class_info = spio.loadmat(r"/media/batman/Windows/linuxshared/xlsa17/data/AWA2/att_splits.mat")

In [5]:
for k in image_info:
    break
    print(k, " -> ", image_info[k])

In [6]:
print("image_info['features'].shape:",image_info['features'].shape)
print(image_info['features'][0])
print("image_info['labels'].shape:",image_info['labels'].shape)
image_info['labels']

class_info['att'] # Class embeddings
class_info['train_loc'] # Seen class images (trg)

class_info['val_loc'] # Seen class images, (validation)
class_info['test_unseen_loc'] # Seen classes, test
class_info['test_seen_loc'] # DON'T NEED NOW

#print()

image_info['features'].shape: (2048, 37322)
[0.12702841 0.         0.38334134 ... 0.0167553  0.12140134 0.2434606 ]
image_info['labels'].shape: (37322, 1)


array([[11061],
       [ 6271],
       [17298],
       ...,
       [ 4014],
       [20615],
       [33202]], dtype=uint16)

In [7]:
# training_images = image_info['features'][:,class_info['train_loc']-1]
# training_labels = image_info['labels'][class_info['train_loc']-1]

# val_images = image_info['features'][:,class_info['val_loc']-1]
# val_labels = image_info['labels'][class_info['val_loc']-1]

# test_images = image_info['features'][:,class_info['test_unseen_loc']-1]
# test_labels = image_info['labels'][class_info['test_unseen_loc']-1]


In [8]:
# print("training_images.shape, training_labels.shape",training_images.shape, training_labels.shape)
# print("val_images.shape, val_labels.shape",training_images.shape, training_labels.shape)
# print("test_images.shape, test_labels.shape",training_images.shape, training_labels.shape)

In [9]:
#print(image_info['features'])
print(image_info['features'].shape)
np.mean(image_info['features'][0])
#[0-1]
#raise

(2048, 37322)


0.30810879527732815

In [10]:
### Prepare data
tr_labels, val_labels, test_labels = [], [], []
# training data
X, Y = [], []
for idx in class_info['train_loc']:
   X.append(l2_norm(image_info['features'].transpose()[idx[0]-1]))
   Y.append(image_info['labels'][idx[0]-1][0]-1)
   if not image_info['labels'][idx[0]-1][0]-1 in tr_labels:
       tr_labels.append(image_info['labels'][idx[0]-1][0]-1)
# val data
val_X, val_Y = [], []
for idx in class_info['val_loc']:
   val_X.append(l2_norm(image_info['features'].transpose()[idx[0]-1]))
   val_Y.append(image_info['labels'][idx[0]-1][0]-1)
   if not image_info['labels'][idx[0]-1][0]-1 in val_labels:
       val_labels.append(image_info['labels'][idx[0]-1][0]-1)
# test data
test_X, test_Y = [], []
for idx in class_info['test_unseen_loc']:
   test_X.append(l2_norm(image_info['features'].transpose()[idx[0]-1]))
   test_Y.append(image_info['labels'][idx[0]-1][0]-1)
   if not image_info['labels'][idx[0]-1][0]-1 in test_labels:
       test_labels.append(image_info['labels'][idx[0]-1][0]-1)

# get all classes embeddings
class_embeddings = class_info['att'].transpose()
# dims of image features and class embeddings
x_dim, yEmb_dim = len(X[0]), len(class_embeddings[0])

print(type(X))
print(len(X))
print(X[0])
print(X[0].shape)
print(len(Y))
print(Y[0:100])
print(np.mean(Y))
#print(Y)
#print(X)

print(class_info['allclasses_names'][0][0])

training_images=np.array(X)
training_labels=np.array(Y)
val_images=np.array(val_X)
val_labels=np.array(val_Y)
test_images=np.array(test_X)
test_labels=np.array(test_Y)
print("training_images.shape, training_labels.shape",training_images.shape, training_labels.shape)
print("val_images.shape, val_labels.shape",training_images.shape, training_labels.shape)
print("test_images.shape, test_labels.shape",training_images.shape, training_labels.shape)

In [11]:
# class split
tr_classes, val_classes, test_classes= [], [], []
print("training classes:")
for idx in tr_labels:
   tr_classes.append(class_info['allclasses_names'][idx-1][0][0])
   print (class_info['allclasses_names'][idx-1][0][0]," ",end="")
print ("\nvalidation classes:")
for idx in val_labels:
   val_classes.append(class_info['allclasses_names'][idx-1][0][0])
   print (class_info['allclasses_names'][idx-1][0][0]," ",end="")
print ("\ntest classes:")
for idx in test_labels:
    test_classes.append(class_info['allclasses_names'][idx-1][0][0])
    print (class_info['allclasses_names'][idx-1][0][0]," ",end="")
    #print(class_info['allclasses_names'][idx-1][0][0], ',',-1][0][0]))
    #print(class_info['allclasses_names'][idx-1][0][0], ',',)

training classes:
dolphin  otter  wolf  polar+bear  raccoon  humpback+whale  ox  horse  antelope  tiger  spider+monkey  grizzly+bear  pig  lion  weasel  dalmatian  bobcat  mouse  squirrel  blue+whale  siamese+cat  moose  hamster  mole  rat  giraffe  buffalo  
validation classes:
killer+whale  seal  beaver  giant+panda  zebra  elephant  chimpanzee  hippopotamus  skunk  leopard  gorilla  rhinoceros  walrus  
test classes:
rabbit  german+shepherd  deer  cow  bat  persian+cat  chihuahua  sheep  fox  collie  

In [12]:
%%script false 
# Alternative way to list classes! 
mj_tr_classes = [class_info['allclasses_names'][idx-1][0][0] for idx in tr_labels]
print("mj_tr_classes:\n", ", ".join(mj_tr_classes))

mj_val_classes = [class_info['allclasses_names'][idx-1][0][0] for idx in val_labels]
print("mj_val_classes:\n", ", ".join(mj_val_classes))

mj_test_classes = [class_info['allclasses_names'][idx-1][0][0] for idx in test_labels]
print("mj_test_classes:\n", ", ".join(mj_test_classes))